In [2]:
import numpy as np
import pandas as pd 
import json

In [3]:
#오류의 원인은 encoding 문제였음.
#endcoding = 'CP949'를 선언하면 제대로 불러온다.

meta = pd.read_csv('0526.csv',encoding='CP949')

meta.head() # 최대 5개를 보여줌.

,name,method,food_id
0,제육볶음,"[{'id': 1, 'name' : 'fry'}, {'id':101,'name' :...",1
1,미역국,"[{'id':2, 'name': 'soup'}, {'id':102,'name':'s...",2
2,소불고기,"[{'id': 1, 'name' : 'fry'}, {'id':104,'name' :...",6
3,회덮밥,"[{'id':3, 'name' : 'raw'}, {'id':102, 'name' :...",4
4,감자채볶음,"[{'id':1,'name':'fry'}, {'id':103,'name':'vege...",5


In [4]:
# meta/name의 이름 변경 
meta = meta.rename(columns={'name':'food_name'}) 

In [5]:
meta.head()

,food_name,method,food_id
0,제육볶음,"[{'id': 1, 'name' : 'fry'}, {'id':101,'name' :...",1
1,미역국,"[{'id':2, 'name': 'soup'}, {'id':102,'name':'s...",2
2,소불고기,"[{'id': 1, 'name' : 'fry'}, {'id':104,'name' :...",6
3,회덮밥,"[{'id':3, 'name' : 'raw'}, {'id':102, 'name' :...",4
4,감자채볶음,"[{'id':1,'name':'fry'}, {'id':103,'name':'vege...",5


In [6]:
# 임의로 생성한 음식별 평점 csv파일을 불러옴 
# 얘는 한글이 없어서 encoding 없이 불러옴 
rating = pd.read_csv('0526r.csv')
rating.head()

#describe() = 간략한 데이터 출력 
rating.describe()

# count = 데이팅 수 
# mean = 평균값
# std = 표준편차
# min = 최소값 
# max = 최대값 

# 아 통계학 공부해야 돼 .............

,user_id,food_id,rating
count,126.000000,126.000000,107.00000
mean,7.500000,5.000000,3.28972
std,4.047221,2.592296,1.63112
min,1.000000,1.000000,1.00000
25%,4.000000,3.000000,1.50000
50%,7.500000,5.000000,3.00000
75%,11.000000,7.000000,5.00000
max,14.000000,9.000000,5.00000


In [7]:
# 데이터를 json으로 가정했기 때문에 string -> int를 위해 numeric을 사용 

meta.food_id = pd.to_numeric(meta.food_id, errors='coerce')
rating.food_id=pd.to_numeric(rating.food_id, errors='coerce')

In [8]:
#함수 생성 
def parse_method(method_str):
    # json.loads는 json 형식의 string을 파이썬의 dictionary로 변환한다.
    # 단 변환할 때 ' -> " 로 바꿔준다.
    method = json.loads(method_str.replace('\'', '"'))
    
    method_list = []
    for g in method:
        #method 값만 남긴 후 method_list에 저장한다.
        method_list.append(g['name'])

    return method_list

# apply :: 각 행에 해당 함수를 적용 
# 이로써 method의 재료와 요리 방법만 남길 수 있는 것.
meta['method'] = meta['method'].apply(parse_method)

meta.head()

,food_name,method,food_id
0,제육볶음,"[fry, poke]",1
1,미역국,"[soup, seafood]",2
2,소불고기,"[fry, beef]",6
3,회덮밥,"[raw, seafood]",4
4,감자채볶음,"[fry, vegetable]",5


In [9]:
# merge = csv 파일끼리 병합하는 것.
# 이때 on = '기준' how = 합치는 방법 
data = pd.merge(rating, meta, on='food_id', how='inner')
data.head()

,user_id,food_id,rating,food_name,method
0,1,1,5.0,제육볶음,"[fry, poke]"
1,2,1,1.0,제육볶음,"[fry, poke]"
2,3,1,3.0,제육볶음,"[fry, poke]"
3,4,1,5.0,제육볶음,"[fry, poke]"
4,5,1,2.0,제육볶음,"[fry, poke]"


In [10]:
# pivot_table = 피벗 테이블 생성 
matrix = data.pivot_table(index='user_id',columns='food_name', values='rating')

matrix.head(20)

# 피봇테이블 생성 및 평가된 데이터 수집 

food_name,가지볶음,감자채볶음,두루치기,미역국,삼겹살볶음,소불고기,제육볶음,초밥,회덮밥
user_id,,,,,,,,,
1,1.0,5.0,NaN,4.0,NaN,5.0,5.0,NaN,1.0
2,3.0,4.0,1.0,5.0,1.0,1.0,1.0,5.0,5.0
3,1.0,3.0,3.0,2.0,3.0,3.0,3.0,4.0,3.0
4,1.0,1.0,5.0,4.0,5.0,5.0,5.0,1.0,5.0
5,2.0,4.0,4.0,5.0,1.0,2.0,2.0,5.0,5.0
6,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
7,5.0,5.0,5.0,1.0,5.0,5.0,5.0,5.0,1.0
8,2.0,5.0,5.0,1.0,1.0,1.0,3.0,3.0,1.0
9,4.0,4.0,NaN,3.0,3.0,3.0,5.0,NaN,2.0


In [11]:
# 상관관계 조사 :: 피어슨 상관관계 
METHOD_WEIGHT = 0.1 # 요리 재료와 방식에 가중치가 들어갈 변수

def pearsonR(s1, s2):
    s1_c = s1 - s1.mean()
    s2_c = s2 - s2.mean()
    return np.sum(s1_c * s2_c)/ np.sqrt(np.sum(s1_c ** 2) *np.sum(s2_c ** 2))
#분모가 클수록 상관관계가 크다.


# n = 추천 개수, similar_method = 같은 재료와 요리 방식이면 점수를 높게 준다.
def recommend(input_food, matrix, n, similar_method=True):
    input_method = meta[meta['food_name']== input_food]['method'].iloc(0)[0]
    #인풋으로 받은 요리의 재료와 종류를 input_method에 저장 
    
    result = []
    for name in matrix.columns : #매트릭스 칼럼을 조회하는 for문
        if name == input_food:
            continue
            #같은 요리를 추천해줄 필요가 없으므로, 입력된 요리와 검색된 요리가
            #같다면 생략한다. 
            
        cor = pearsonR(matrix[input_food], matrix[name])
        
        #meta[food_name]과 name과 비교를 해야 온전한 값이 들어가는데, 
        #input_food를 사용하여 입력된 method만 나타났던 것
        #rating 값도 온전히 나오는 것을 확인할 수 있다.
        #하하하하
        #결국 중요한 건 데이터의 크기. 
        #데이터를 작게 만들려면 ... 상관관계를 잘 파악하고 점수를 입력해야할 듯?
        
        if similar_method and len(input_method) > 0:
            temp_method = meta[meta['food_name']== name]['method'].iloc(0)[0]
            
            #배열을 비교하여 똑같은 요소가 있으면 True를 반환한다.
            #np.isin(): 배열을 비교하여 똑같은 요소가 있으면 True를 반환한다.
            same_count = np.sum(np.isin(input_method, temp_method))
            cor += (METHOD_WEIGHT * same_count )
        # 배열에 nan포함 여부 
        # non일 경우 continue를 적용하여 넘어간다.
        
        if np.isnan(cor):
            continue
        else:
            result.append((name, '{:2f}'.format(cor), temp_method))
    #lambda x:x == def temp(x) :
    #               return x
    # rating 값이 높은 순서대로 내림차순으로 정렬함.
    result.sort(key=lambda r: r[1], reverse=True)
    
    # n개만 반환.
    return result[:n]
    

In [12]:
recommend_result = recommend('제육볶음', matrix, 5, similar_method=True)
pd.DataFrame(recommend_result, columns = ['Food_Name','Rating', 'Method'])

## 이것이 Correlation 상관관계 
# 아니 근데 왜 쟤 poke로 나오지? 결과의 Method가 아니라 입력의 Method가 출력되는 문제였음.
# 레이팅 값 조절 완료 
# 결국 id와 그에 따른 rating 값으로 추천을 하므로, rating 입력이 중요함.


,Food_Name,Rating,Method
0,소불고기,0.959129,"[fry, beef]"
1,두루치기,0.917428,"[fry, poke]"
2,삼겹살볶음,0.855373,"[fry, poke]"
3,가지볶음,0.310578,"[fry, vegetable]"
4,감자채볶음,0.262547,"[fry, vegetable]"
